---
# Spatial Pooler

---
- encoding 된 data 를 입력받아 희소분포표상(SDRs) 를 생성한다.
- 시간 t 는 특정시간을 나타내는데, 모든 스텝의 정보를 저장하지말고 prev, cur 형태로만 저장하면 된다.
- input 은 placeholder 로 간주하면 된다.
---

In [129]:
import numpy as np
import math

In [144]:
class SpatialPooler:
    def __init__(self, encoded_data, w=5, h=5, synapse_count=4):
        ''' layer 의 속성을 결정하는 hyper parameters '''
        self.encoded_data = encoded_data            # encoder 에서 encoding 되어 들어온 입력 데이터
        self.column_w = w                           # column 가로 크기
        self.column_h = h                           # column 세로 크기
        self.total_columns = self.column_w * self.column_h                  # 총 컬럼의 수
        self.synapses = synapse_count               # 각 컬럼 당 아랫단 계층과 연결되는 synapse 수
        self.min_overlap = 5                        # 발화 하기 위한 최소한의 overlap count
        self.minLocalActivity = 5                   # 발화할 임계치
        self.desiredLocalActivity = 5               # 발화할 column 수
        self.connectedPerm = .5                     # synapse permanence 임계치
        
        ''' layer 에 존재하는 column 과 학습 대상 data 들  '''
        self.overlap = np.zeros([self.column_w, self.column_h])                            # 입력과 overlap 되는 데이터 카운트. 컬럼 당 하나. 1차원으로 만들어도 됨.
        self.potentialSynapses = np.ones([self.column_w, self.column_h, self.synapses])    # 각 컬럼 당 입력 layer 와 연결된 모든 잠재적 시냅스의 permanence
        self.boost = np.ones([self.column_w, self.column_h])                               # 가중치
        self.activateColumns = []        # 승리한 컬럼 리스트 - bool
    
    def compute_SDR(self):
        ''' 1. 겹치게 하기 '''
        for w in range(self.column_w):
            for h in range(self.column_h):
                self.overlap[w, h] = 0
                
                connectedSynapses = np.where(self.potentialSynapses[w, h] > self.connectedPerm)
                
                for s in connectedSynapses[0]:
                    self.overlap[w, h] += self.encoded_data[s + w*self.column_w + h]  
                
                if(self.overlap[w, h] < self.min_overlap):
                    self.overlap[w, h] = 0
                else:
                    self.overlap[w, h] *=  self.boost[w, h]
                                
        ''' 2. 억제 '''        
        for w in range(self.column_w):
            for h in range(self.column_h):
                self.minLocalActivity = self.kthScore(0, self.desiredLocalActivity)

                if(self.overlap[w, h] > self.minLocalActivity):
                    self.activateColumns.append((w, h))
                            
        ''' 3. 학습 '''
        for w, h in self.activateColumns:
            order = 0
            for perm in self.potentialSynapses[w, h]:
                if(perm > self.connectedPerm):
                    perm = np.clip(perm+.1, 0, 1)
                    self.potentialSynapses[w, h, order]
                else:
                    perm = np.clip(perm-.1, 0, 1)
                    self.potentialSynapses[w, h, order]

                order += 1
    
    def kthScore(self, column, desired_kth):
        #np.sort(self.overlap)
        return 1

In [146]:
test_data = np.arange(40)
s = SpatialPooler(test_data)
s.compute_SDR()

In [167]:
arr = np.arange(24).reshape([2,3,-1])
order = 0
for i in a[1,2]:
    i = 0
    arr[1,2,order] = i
    order += 1
arr

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [ 0,  0, 22, 23]]])